In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import IPython
import os
import time
import re
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from pearlsim.ml_utilities import *

# Extract data from Serpent detector files
We can use the read_det_file function from ml_utilities to parse all of the detector files we have in the training data directory to create one pair of unified features/target dataframes.

In [7]:
file_names = os.listdir("training_data/")
all_features = pd.DataFrame([])
all_targets = pd.DataFrame([])
for name in file_names:
    if "det" in name:
        features, targets, avg_uncertainty = read_det_file("training_data/"+name)
        print(f"File {name} has an average {round(avg_uncertainty*100,2)}% uncertainty.")
        all_features = pd.concat([all_features, features])
        all_targets = pd.concat([all_targets, targets])
all_features.to_csv("training_data/current_data.csv")
all_targets.to_csv("training_data/current_target.csv")


File gFHR_core_1.serpent_det1.m has an average 44.83% uncertainty.


Our features include the radius and height of the pebble-based detector. It also includes every bin of the core flux map. The gFHR model is divided into 12 energy groups and has 4 radial divisions, each divided into 10 separate axial zones. Normally you would need to volume-weight these flux values, and deal with the somewhat complicated indexing scheme to sort out which is which. We don't need to bother here, since they're all going to be standardized and the model cares not about where the bin is.

In [69]:
all_features.iloc[0:10]

,radius,height,bin1,bin2,bin3,bin4,bin5,bin6,bin7,bin8,...,bin471,bin472,bin473,bin474,bin475,bin476,bin477,bin478,bin479,bin480
0,81.0486,178.1140,0.0,0.0,0.0,0.0,4.842880e+17,1.321370e+18,1.822170e+18,1.956790e+18,...,4.674090e+18,4.884910e+18,5.613250e+17,1.508330e+18,1.994230e+18,1.850610e+18,3.740820e+15,1.845760e+16,2.816800e+16,1.054690e+16
1,57.7707,151.1960,0.0,0.0,0.0,0.0,4.842880e+17,1.321370e+18,1.822170e+18,1.956790e+18,...,4.674090e+18,4.884910e+18,5.613250e+17,1.508330e+18,1.994230e+18,1.850610e+18,3.740820e+15,1.845760e+16,2.816800e+16,1.054690e+16
2,91.4887,326.0362,0.0,0.0,0.0,0.0,4.842880e+17,1.321370e+18,1.822170e+18,1.956790e+18,...,4.674090e+18,4.884910e+18,5.613250e+17,1.508330e+18,1.994230e+18,1.850610e+18,3.740820e+15,1.845760e+16,2.816800e+16,1.054690e+16
3,82.7382,290.7679,0.0,0.0,0.0,0.0,4.842880e+17,1.321370e+18,1.822170e+18,1.956790e+18,...,4.674090e+18,4.884910e+18,5.613250e+17,1.508330e+18,1.994230e+18,1.850610e+18,3.740820e+15,1.845760e+16,2.816800e+16,1.054690e+16
4,93.6223,73.5450,0.0,0.0,0.0,0.0,4.842880e+17,1.321370e+18,1.822170e+18,1.956790e+18,...,4.674090e+18,4.884910e+18,5.613250e+17,1.508330e+18,1.994230e+18,1.850610e+18,3.740820e+15,1.845760e+16,2.816800e+16,1.054690e+16
5,52.1650,84.2520,0.0,0.0,0.0,0.0,4.842880e+17,1.321370e+18,1.822170e+18,1.956790e+18,...,4.674090e+18,4.884910e+18,5.613250e+17,1.508330e+18,1.994230e+18,1.850610e+18,3.740820e+15,1.845760e+16,2.816800e+16,1.054690e+16
6,68.1323,246.0250,0.0,0.0,0.0,0.0,4.842880e+17,1.321370e+18,1.822170e+18,1.956790e+18,...,4.674090e+18,4.884910e+18,5.613250e+17,1.508330e+18,1.994230e+18,1.850610e+18,3.740820e+15,1.845760e+16,2.816800e+16,1.054690e+16
7,93.1488,149.5170,0.0,0.0,0.0,0.0,4.842880e+17,1.321370e+18,1.822170e+18,1.956790e+18,...,4.674090e+18,4.884910e+18,5.613250e+17,1.508330e+18,1.994230e+18,1.850610e+18,3.740820e+15,1.845760e+16,2.816800e+16,1.054690e+16
8,84.4764,244.5620,0.0,0.0,0.0,0.0,4.842880e+17,1.321370e+18,1.822170e+18,1.956790e+18,...,4.674090e+18,4.884910e+18,5.613250e+17,1.508330e+18,1.994230e+18,1.850610e+18,3.740820e+15,1.845760e+16,2.816800e+16,1.054690e+16
9,45.3588,364.6420,0.0,0.0,0.0,0.0,4.842880e+17,1.321370e+18,1.822170e+18,1.956790e+18,...,4.674090e+18,4.884910e+18,5.613250e+17,1.508330e+18,1.994230e+18,1.850610e+18,3.740820e+15,1.845760e+16,2.816800e+16,1.054690e+16


Meanwhile, the target variables represent the neutron current going into the pebbles above, divided into the same 12 energy bins. There's a tradeoff between how fine our energy grid is, and how long it takes for the Serpent model to run. Every time you add a new bin, you need to simulate more particles to get sufficient statistics. So, our goal is to maximize accuracy using as few bins as possible.

In [70]:
all_targets.iloc[0:10]

,0.000000e+00,3.000000e-08,5.800000e-08,1.400000e-07,2.800000e-07,3.500000e-07,6.250000e-07,4.000000e-06,4.805200e-05,5.530000e-03,8.210000e-01,2.231000e+00
0,2.990610e+13,1.509400e+14,5.712310e+14,1.109120e+15,2.100110e+14,1.802850e+14,5.434490e+14,7.206980e+14,1.171560e+15,1.533240e+15,2.412640e+14,1.193780e+14
1,1.175830e+14,1.503580e+14,6.011330e+14,8.711530e+14,1.510600e+14,4.505320e+14,4.838480e+14,5.379950e+14,1.292430e+15,1.258730e+15,1.795970e+14,1.797880e+14
2,1.500560e+14,1.516900e+14,6.632820e+14,6.599860e+14,6.040250e+13,3.015640e+14,2.380500e+14,5.708930e+14,8.673570e+14,9.640530e+14,6.015700e+13,1.787810e+14
3,6.074220e+13,2.418670e+14,4.797750e+14,7.865030e+14,2.114810e+14,3.297250e+14,6.304340e+14,7.539350e+14,1.206990e+15,9.563150e+14,1.504240e+14,9.069430e+13
4,8.947420e+13,6.004230e+13,2.095480e+14,2.404720e+14,1.194590e+14,5.945690e+13,1.508320e+14,1.826630e+14,2.401050e+14,3.310770e+14,6.019230e+13,2.992770e+13
5,2.950450e+13,2.105400e+14,4.484400e+14,3.331050e+14,5.997480e+13,1.507330e+14,1.503540e+14,4.775980e+14,8.417780e+14,5.712610e+14,2.421340e+14,8.882490e+13
6,9.195050e+13,2.099640e+14,6.625630e+14,1.025070e+15,1.812910e+14,3.917210e+14,6.314170e+14,9.875660e+14,1.318640e+15,1.346790e+15,3.613620e+14,6.065130e+13
7,2.090090e+14,6.050000e+13,5.407110e+14,8.997840e+14,1.496130e+14,2.090720e+14,3.621340e+14,2.688810e+14,1.024560e+15,1.108750e+15,3.910220e+14,1.503610e+14
8,6.010570e+13,9.046960e+13,5.104570e+14,5.421830e+14,1.204680e+14,1.531660e+14,3.884190e+14,6.311450e+14,1.748390e+15,1.051830e+15,3.913710e+14,8.978980e+13
9,6.087510e+13,1.195970e+14,2.112990e+14,6.934100e+14,1.502210e+14,9.058230e+13,1.208060e+14,2.099660e+14,4.499100e+14,7.232850e+14,2.099650e+14,6.001770e+13


# Data Standardization
Simple standardization is performed here along each column. You might want to consider looking into log standardization, but it didn't seem to help much for me.

In [114]:
train_split = 0.8
np.random.seed(42)

def standardize(raw_data, mean=None, std=None, axis=0):
    if mean is None:
        mean = np.mean(raw_data, axis = axis)
    if std is None:
        std = np.std(raw_data, axis = axis)
        std[ std==0 ] = 0.1
    result = (raw_data - mean) / std
    return result, mean, std

def unstandardize(standardized_data, mean, std):
    raw_data = (standardized_data*std)+mean
    return raw_data

#log_features = all_features#.apply(lambda x: np.log10(x + 1))
#log_targets = all_targets
#log_targets.iloc[:,2:] = all_targets.iloc[:,2:]#.apply(lambda x: np.log10(x + 1))

num_data = len(all_features)
training_size = int(num_data*train_split)
testing_size = num_data - training_size
data_indices = np.arange(num_data)
training_indices = np.random.choice(num_data, training_size, replace=False)
testing_indices = data_indices[np.in1d(data_indices, training_indices, invert=True)]

training_data, data_mean, data_std = standardize(all_features.iloc[training_indices])
training_target, target_mean, target_std = standardize(all_targets.iloc[training_indices])
testing_data, _, _  = standardize(all_features.iloc[testing_indices], mean=data_mean, std=data_std)
testing_target, _, _  = standardize(all_targets.iloc[testing_indices], mean=target_mean, std=target_std)

print(np.shape(training_data))
print(np.shape(training_target))
print(np.shape(testing_data))
print(np.shape(testing_target))

(800, 482)
(800, 12)
(200, 482)
(200, 12)


# Model Training
I threw together a quick RFR model and got some results. You're free to change to any other type of model, as long as its something I can save and load into other modules. Things to try:
- Properly using cross validation
- Tuning the hyper parameters
- Trying a different model, probably a neural net

In [115]:
best_params = {'max_depth': 10, 
               'n_estimators': 1000, 
               'n_jobs': 14,}
rfr_model = RandomForestRegressor(random_state=0)
rfr_model.set_params(**best_params)
rfr_model.fit(training_data, training_target)
rfr_model_test_score = rfr_model.score(testing_data, testing_target)
print(f"RFR score: {rfr_model_test_score}")

RFR score: -0.545159239552131
